In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
print(module_path)

/Users/revilooliver/Documents/quantum_computing/research/pauli_sandwitching/cut4mitigation


In [3]:
# author: Ji Liu email: ji.liu@anl.gov

import itertools, numpy
import circuit_cutter
import mlrecon_methods as ml
import numpy as np
import mthree

import qiskit
from qiskit import *
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute, transpile

from qiskit.transpiler import PassManager

from qiskit.converters import circuit_to_dag
from qiskit.visualization import dag_drawer, plot_histogram
from qiskit.compiler import assemble

from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview

import qiskit.providers.aer.noise as noise
from qiskit.providers.aer.noise import NoiseModel

from utils.utils import filter_results, dict_to_list, H_distance, total_counts, norm_dict
from utils.vqe_utils import read_from_file, MeasureCircuit, find_commute_groups, evaluation, apply_checking_circuit
from mlrecon_methods import run_circuits, collect_fragment_circuits

In [4]:
qiskit.__qiskit_version__

{'qiskit-terra': '0.22.2', 'qiskit-aer': '0.11.1', 'qiskit-ignis': '0.7.1', 'qiskit-ibmq-provider': '0.19.2', 'qiskit': '0.39.2', 'qiskit-nature': '0.5.1', 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}

In [5]:
numpy.set_printoptions(linewidth = 200)

simulation_backend = "qasm_simulator"

In [6]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-ornl', group='ornl', project='chm185')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_montreal') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_toronto') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_kolkata') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_mumbai') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_guadalupe') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q-ornl', grou

In [7]:
device = provider.get_backend('ibm_auckland')
device

<IBMQBackend('ibm_auckland') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>

In [8]:
device.configuration().to_dict()['max_shots']

100000

In [9]:
from qiskit import IBMQ, Aer
from qiskit.providers.aer.noise import NoiseModel
noise_model = NoiseModel.from_backend(device, gate_error = False, thermal_relaxation= False)
print(noise_model)

NoiseModel:
  Basis gates: ['cx', 'id', 'reset', 'rz', 'sx', 'x']
  Instructions with noise: ['measure']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
  Specific qubit errors: [('measure', (0,)), ('measure', (1,)), ('measure', (2,)), ('measure', (3,)), ('measure', (4,)), ('measure', (5,)), ('measure', (6,)), ('measure', (7,)), ('measure', (8,)), ('measure', (9,)), ('measure', (10,)), ('measure', (11,)), ('measure', (12,)), ('measure', (13,)), ('measure', (14,)), ('measure', (15,)), ('measure', (16,)), ('measure', (17,)), ('measure', (18,)), ('measure', (19,)), ('measure', (20,)), ('measure', (21,)), ('measure', (22,)), ('measure', (23,)), ('measure', (24,)), ('measure', (25,)), ('measure', (26,))]


In [10]:
noisy_simulator = provider.get_backend('ibmq_qasm_simulator')

In [11]:
front_layer = QuantumCircuit.from_qasm_file("qasm/vqe_LiH_front.qasm")

In [12]:
mid_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_mid.qasm")

In [13]:
end_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_end.qasm")

In [14]:
orign_circuit = front_layer + mid_layer + end_layer

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_23579/743480212.py:1: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  orign_circuit = front_layer + mid_layer + end_layer


In [15]:
orign_circuit.draw()

┌───┐       ┌───────────────┐┌───────────────┐   ┌───────────────┐»
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├─■─┤ Ry(0.0082611) ├»
            ├───┤       └┬──────────────┤├───────────────┤ │ └───────────────┘»
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─■─────────■────────»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘           │        »
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├────────────────────────────■────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                                     »
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├──────────────────────────────────────»
     └─────────────────┘  └────────────┘                                      »
q_4: ─────────────────────────────────────────────────────────────────────────»
                                                                              »
«      ┌────────────┐                                    
«q_0: ─┤ Rz(1.3184) ├────────────────────────────────────
«     ┌┴────────────┴┐   ┌────────────┐                  
«q_1: ┤ Ry(0.013556) ├───┤ Rz(1.3353) ├──────────────────
«     └──────────────┘  ┌┴────────────┴┐ ┌──────────────┐
«q_2: ───────■──────────┤ Ry(0.051652) ├─┤ Rz(-0.26461) ├
«            │        ┌─┴──────────────┴┐└┬────────────┬┘
«q_3: ───────■────────┤ Ry(-4.5556e-05) ├─┤ Rz(1.2199) ├─
«                     └─────────────────┘ └────────────┘ 
«q_4: ───────────────────────────────────────────────────
«

In [16]:
#checking circuit

In [17]:
phase_y_front = [0.008239051732521298, 0.013635804359443573, 0.05162548888524782, -2.1999413989625588e-05]
phase_z_front = [-0.010143905866716232, -0.028142425222093263, 0.008659821134257232, 1.4378429162081177]
phase_y_end = [0.008261085257040952, 0.013555948647231175, 0.0516520269898158, -4.555585169085397e-05]
phase_z_end = [1.3183908530236275, 1.3352666291100785, -0.26460520551963923, 1.2198754282688027]
x_front = [True, True, False, False]

In [18]:
qubit_id = 2
circuit = QuantumCircuit(5)
apply_checking_circuit(circuit, [qubit_id], [4], side = 'front', phase_y = phase_y_front[qubit_id], phase_z = phase_z_front[qubit_id], x = x_front[qubit_id])
circuit += orign_circuit
apply_checking_circuit(circuit, [qubit_id], [4], side = 'end', phase_y = phase_y_end[qubit_id], phase_z = phase_z_end[qubit_id])

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_23579/2421900640.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  circuit += orign_circuit


In [19]:
circuit.draw()

┌───┐       ┌───────────────┐┌───────────────┐                  »
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├────────■─────────»
            ├───┤       └┬──────────────┤├───────────────┤        │         »
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├────────■─────────»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘┌────────────────┐»
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├────────■────────┤ Rz(-0.0086598) ├»
     ┌─┴──────────────┴┐└─┬────────────┬┘        │        └────────────────┘»
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├─────────┼──────────────────────────»
     └──────┬───┬──────┘  └────────────┘         │                          »
q_4: ───────┤ H ├────────────────────────────────■──────────────────────────»
            └───┘                                                           »
«     ┌───────────────┐ ┌────────────┐                                     »
«q_0: ┤ Ry(0.0082611) ├─┤ Rz(1.3184) ├─────────────────────────────────────»
«     └───────────────┘ └────────────┘                     ┌──────────────┐»
«q_1: ───────────────────────────────────────────────────■─┤ Ry(0.013556) ├»
«     ┌───────────────┐┌──────────────┐┌───────────────┐ │ └──────────────┘»
«q_2: ┤ Ry(-0.051625) ├┤ Ry(0.051625) ├┤ Rz(0.0086598) ├─■────────■────────»
«     └───────────────┘└──────────────┘└───────────────┘          │        »
«q_3: ────────────────────────────────────────────────────────────■────────»
«                                                                          »
«q_4: ─────────────────────────────────────────────────────────────────────»
«                                                                          »
«                                                                           »
«q_0: ──────────────────────────────────────────────────────────────────────»
«        ┌────────────┐                                                     »
«q_1: ───┤ Rz(1.3353) ├─────────────────────────────────────────────────────»
«       ┌┴────────────┴┐ ┌──────────────┐┌─────────────┐┌───────────────┐   »
«q_2: ──┤ Ry(0.051652) ├─┤ Rz(-0.26461) ├┤ Rz(0.26461) ├┤ Ry(-0.051652) ├─■─»
«     ┌─┴──────────────┴┐└┬────────────┬┘└─────────────┘└───────────────┘ │ »
«q_3: ┤ Ry(-4.5556e-05) ├─┤ Rz(1.2199) ├──────────────────────────────────┼─»
«     └─────────────────┘ └────────────┘                                  │ »
«q_4: ────────────────────────────────────────────────────────────────────■─»
«                                                                           »
«                                     
«q_0: ────────────────────────────────
«                                     
«q_1: ────────────────────────────────
«     ┌──────────────┐┌──────────────┐
«q_2: ┤ Ry(0.051652) ├┤ Rz(-0.26461) ├
«     └──────────────┘└──────────────┘
«q_3: ────────────────────────────────
«          ┌───┐                      
«q_4: ─────┤ H ├──────────────────────
«          └───┘

In [20]:
pauli_list = read_from_file('LiH_hamiltonian_1.5.txt')
pauli_commute = find_commute_groups(pauli_list)

In [21]:
#generate individual measurment circuits that changes the basis
meas_qcs = []
for i in range(0, len(pauli_commute)):
    temp_qc = MeasureCircuit(pauli_commute[i], num_qubits = 4,num_qargs = 5)
    meas_qcs.append(temp_qc)
    print(temp_qc)

YYYY
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_4: ────────────────
                     
XZYY
      ┌────────────┐ 
q_0: ─┤ U(π/2,0,π) ├─
      └────────────┘ 
q_1: ────────────────
     ┌──────────────┐
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_4: ────────────────
                     
YYXZ
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_2: ─┤ U(π/2,0,π) ├─
      └────────────┘ 
q_3: ────────────────
                     
q_4: ────────────────
                     
XXYY
      ┌────────────┐ 
q_0: ─┤ U(π/2,0,π) ├─
      ├────────────┤ 
q_1: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_4: ────────────────
                     
YYXX
     ┌─

In [22]:
circuit_list = []
for meas in meas_qcs:
    temp_circuit = circuit.copy()
    temp_circuit += meas
    circuit_list.append(temp_circuit)

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_23579/1717817760.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  temp_circuit += meas


In [23]:
circuit_list[-2].draw()

┌───┐       ┌───────────────┐┌───────────────┐                  »
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├────────■─────────»
            ├───┤       └┬──────────────┤├───────────────┤        │         »
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├────────■─────────»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘┌────────────────┐»
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├────────■────────┤ Rz(-0.0086598) ├»
     ┌─┴──────────────┴┐└─┬────────────┬┘        │        └────────────────┘»
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├─────────┼──────────────────────────»
     └──────┬───┬──────┘  └────────────┘         │                          »
q_4: ───────┤ H ├────────────────────────────────■──────────────────────────»
            └───┘                                                           »
«     ┌───────────────┐ ┌────────────┐                                     »
«q_0: ┤ Ry(0.0082611) ├─┤ Rz(1.3184) ├─────────────────────────────────────»
«     └───────────────┘ └────────────┘                     ┌──────────────┐»
«q_1: ───────────────────────────────────────────────────■─┤ Ry(0.013556) ├»
«     ┌───────────────┐┌──────────────┐┌───────────────┐ │ └──────────────┘»
«q_2: ┤ Ry(-0.051625) ├┤ Ry(0.051625) ├┤ Rz(0.0086598) ├─■────────■────────»
«     └───────────────┘└──────────────┘└───────────────┘          │        »
«q_3: ────────────────────────────────────────────────────────────■────────»
«                                                                          »
«q_4: ─────────────────────────────────────────────────────────────────────»
«                                                                          »
«                                                                           »
«q_0: ──────────────────────────────────────────────────────────────────────»
«        ┌────────────┐                                                     »
«q_1: ───┤ Rz(1.3353) ├─────────────────────────────────────────────────────»
«       ┌┴────────────┴┐ ┌──────────────┐┌─────────────┐┌───────────────┐   »
«q_2: ──┤ Ry(0.051652) ├─┤ Rz(-0.26461) ├┤ Rz(0.26461) ├┤ Ry(-0.051652) ├─■─»
«     ┌─┴──────────────┴┐└┬────────────┬┘└┬────────────┤└───────────────┘ │ »
«q_3: ┤ Ry(-4.5556e-05) ├─┤ Rz(1.2199) ├──┤ U(π/2,0,π) ├──────────────────┼─»
«     └─────────────────┘ └────────────┘  └────────────┘                  │ »
«q_4: ────────────────────────────────────────────────────────────────────■─»
«                                                                           »
«                                     
«q_0: ────────────────────────────────
«                                     
«q_1: ────────────────────────────────
«     ┌──────────────┐┌──────────────┐
«q_2: ┤ Ry(0.051652) ├┤ Rz(-0.26461) ├
«     └──────────────┘└──────────────┘
«q_3: ────────────────────────────────
«          ┌───┐                      
«q_4: ─────┤ H ├──────────────────────
«          └───┘

In [24]:
# cuts = []
# for i in range(0, 1):
#     if i == 0 or i == qubits - 1:
#         cut = [(circuit.qubits[i], 5),(circuit.qubits[i], 6)]
#     else:
#         cut = [(circuit.qubits[i], 5),(circuit.qubits[i], 4)]
#     cuts += cut

In [25]:
cuts = [(circuit.qubits[2], 5),(circuit.qubits[2], 11)]

In [26]:
fragments_list = []
wire_path_map_list = []
for temp_qc in circuit_list:
    print(temp_qc)
    fragments, wire_path_map = circuit_cutter.cut_circuit(temp_qc, cuts)
    fragments_list.append(fragments)
    wire_path_map_list.append(wire_path_map)

            ┌───┐       ┌───────────────┐┌───────────────┐                  »
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├────────■─────────»
            ├───┤       └┬──────────────┤├───────────────┤        │         »
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├────────■─────────»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘┌────────────────┐»
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├────────■────────┤ Rz(-0.0086598) ├»
     ┌─┴──────────────┴┐└─┬────────────┬┘        │        └────────────────┘»
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├─────────┼──────────────────────────»
     └──────┬───┬──────┘  └────────────┘         │                          »
q_4: ───────┤ H ├────────────────────────────────■──────────────────────────»
            └───┘                                                           »
«     ┌───────────────┐ ┌────────────┐  ┌──────────────┐                   »
«q_0: ┤ Ry(0.0082611) ├─┤ Rz(1.3184) ├──┤ U(π/2,0,π/2) ├─────────

            ┌───┐       ┌───────────────┐┌───────────────┐                  »
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├────────■─────────»
            ├───┤       └┬──────────────┤├───────────────┤        │         »
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├────────■─────────»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘┌────────────────┐»
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├────────■────────┤ Rz(-0.0086598) ├»
     ┌─┴──────────────┴┐└─┬────────────┬┘        │        └────────────────┘»
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├─────────┼──────────────────────────»
     └──────┬───┬──────┘  └────────────┘         │                          »
q_4: ───────┤ H ├────────────────────────────────■──────────────────────────»
            └───┘                                                           »
«     ┌───────────────┐ ┌────────────┐  ┌──────────────┐                   »
«q_0: ┤ Ry(0.0082611) ├─┤ Rz(1.3184) ├──┤ U(π/2,0,π/2) ├─────────

            ┌───┐       ┌───────────────┐┌───────────────┐                  »
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├────────■─────────»
            ├───┤       └┬──────────────┤├───────────────┤        │         »
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├────────■─────────»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘┌────────────────┐»
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├────────■────────┤ Rz(-0.0086598) ├»
     ┌─┴──────────────┴┐└─┬────────────┬┘        │        └────────────────┘»
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├─────────┼──────────────────────────»
     └──────┬───┬──────┘  └────────────┘         │                          »
q_4: ───────┤ H ├────────────────────────────────■──────────────────────────»
            └───┘                                                           »
«     ┌───────────────┐ ┌────────────┐                                     »
«q_0: ┤ Ry(0.0082611) ├─┤ Rz(1.3184) ├───────────────────────────

            ┌───┐       ┌───────────────┐┌───────────────┐                  »
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├────────■─────────»
            ├───┤       └┬──────────────┤├───────────────┤        │         »
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├────────■─────────»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘┌────────────────┐»
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├────────■────────┤ Rz(-0.0086598) ├»
     ┌─┴──────────────┴┐└─┬────────────┬┘        │        └────────────────┘»
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├─────────┼──────────────────────────»
     └──────┬───┬──────┘  └────────────┘         │                          »
q_4: ───────┤ H ├────────────────────────────────■──────────────────────────»
            └───┘                                                           »
«     ┌───────────────┐ ┌────────────┐   ┌────────────┐                    »
«q_0: ┤ Ry(0.0082611) ├─┤ Rz(1.3184) ├───┤ U(π/2,0,π) ├──────────

            ┌───┐       ┌───────────────┐┌───────────────┐                  »
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├────────■─────────»
            ├───┤       └┬──────────────┤├───────────────┤        │         »
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├────────■─────────»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘┌────────────────┐»
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├────────■────────┤ Rz(-0.0086598) ├»
     ┌─┴──────────────┴┐└─┬────────────┬┘        │        └────────────────┘»
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├─────────┼──────────────────────────»
     └──────┬───┬──────┘  └────────────┘         │                          »
q_4: ───────┤ H ├────────────────────────────────■──────────────────────────»
            └───┘                                                           »
«     ┌───────────────┐ ┌────────────┐   ┌────────────┐                    »
«q_0: ┤ Ry(0.0082611) ├─┤ Rz(1.3184) ├───┤ U(π/2,0,π) ├──────────

            ┌───┐       ┌───────────────┐┌───────────────┐                  »
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├────────■─────────»
            ├───┤       └┬──────────────┤├───────────────┤        │         »
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├────────■─────────»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘┌────────────────┐»
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├────────■────────┤ Rz(-0.0086598) ├»
     ┌─┴──────────────┴┐└─┬────────────┬┘        │        └────────────────┘»
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├─────────┼──────────────────────────»
     └──────┬───┬──────┘  └────────────┘         │                          »
q_4: ───────┤ H ├────────────────────────────────■──────────────────────────»
            └───┘                                                           »
«     ┌───────────────┐ ┌────────────┐                                     »
«q_0: ┤ Ry(0.0082611) ├─┤ Rz(1.3184) ├───────────────────────────

            ┌───┐       ┌───────────────┐┌───────────────┐                  »
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├────────■─────────»
            ├───┤       └┬──────────────┤├───────────────┤        │         »
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├────────■─────────»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘┌────────────────┐»
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├────────■────────┤ Rz(-0.0086598) ├»
     ┌─┴──────────────┴┐└─┬────────────┬┘        │        └────────────────┘»
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├─────────┼──────────────────────────»
     └──────┬───┬──────┘  └────────────┘         │                          »
q_4: ───────┤ H ├────────────────────────────────■──────────────────────────»
            └───┘                                                           »
«     ┌───────────────┐ ┌────────────┐                                     »
«q_0: ┤ Ry(0.0082611) ├─┤ Rz(1.3184) ├───────────────────────────

In [27]:
print(wire_path_map_list[0])

{Qubit(QuantumRegister(5, 'q'), 0): ((0, Qubit(QuantumRegister(4, 'q'), 2)),), Qubit(QuantumRegister(5, 'q'), 1): ((0, Qubit(QuantumRegister(4, 'q'), 3)),), Qubit(QuantumRegister(5, 'q'), 2): ((1, Qubit(QuantumRegister(3, 'q'), 0)), (0, Qubit(QuantumRegister(4, 'q'), 0)), (1, Qubit(QuantumRegister(3, 'q'), 1))), Qubit(QuantumRegister(5, 'q'), 3): ((0, Qubit(QuantumRegister(4, 'q'), 1)),), Qubit(QuantumRegister(5, 'q'), 4): ((1, Qubit(QuantumRegister(3, 'q'), 2)),)}


In [28]:
total_variants = ml.fragment_variants(wire_path_map_list[0])

In [29]:
total_variants

24

In [30]:
shots = 10000 * total_variants

In [31]:
fragments_list[0][0].draw()

┌──────────────┐ ┌───────────────┐                                     »
q_0: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├─────────────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                                     »
q_1: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├──────────────────────────────────────»
     └──────┬───┬──────┘┌─┴────────────┴┐┌───────────────┐   ┌───────────────┐»
q_2: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├─■─┤ Ry(0.0082611) ├»
            ├───┤       └┬──────────────┤├───────────────┤ │ └───────────────┘»
q_3: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─■──────────────────»
            └───┘        └──────────────┘└───────────────┘                    »
«                          ┌──────────────┐ ┌──────────────┐                
«q_0: ─■────────■──────────┤ Ry(0.051652) ├─┤ Rz(-0.26461) ├────────────────
«      │        │        ┌─┴──────────────┴┐└┬────────────┬┘┌──────────────┐
«q_1: ─┼────────■────────┤ Ry(-4.5556e-05) ├─┤ Rz(1.2199) ├─┤ U(π/2,0,π/2) ├
«      │  ┌────────────┐ └─┬──────────────┬┘ └────────────┘ └──────────────┘
«q_2: ─┼──┤ Rz(1.3184) ├───┤ U(π/2,0,π/2) ├─────────────────────────────────
«      │ ┌┴────────────┴┐  └┬────────────┬┘ ┌──────────────┐                
«q_3: ─■─┤ Ry(0.013556) ├───┤ Rz(1.3353) ├──┤ U(π/2,0,π/2) ├────────────────
«        └──────────────┘   └────────────┘  └──────────────┘

In [32]:
fragment_cuts_list = []
for index in range(0, len(fragments_list)):
    for idx, fragment in enumerate(fragments_list[index]):
        print(f"fragment {idx}:")
        print(fragment)
        print()
    fragment_cuts = ml.fragment_cuts(wire_path_map_list[index])
    fragment_cuts_list.append(fragment_cuts)


fragment 0:
       ┌──────────────┐ ┌───────────────┐                                     »
q_0: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├─────────────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                                     »
q_1: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├──────────────────────────────────────»
     └──────┬───┬──────┘┌─┴────────────┴┐┌───────────────┐   ┌───────────────┐»
q_2: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├─■─┤ Ry(0.0082611) ├»
            ├───┤       └┬──────────────┤├───────────────┤ │ └───────────────┘»
q_3: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─■──────────────────»
            └───┘        └──────────────┘└───────────────┘                    »
«                          ┌──────────────┐ ┌──────────────┐                
«q_0: ─■────────■──────────┤ Ry(0.051652) ├─┤ Rz(-0.26461) ├────────────────
«      │        │        ┌─┴──────────────┴┐└┬────────────┬┘┌──────────────┐
«q_1: ─┼────────■────────┤ Ry(-4.5556

     ┌──────────────┐┌───────────────┐   ┌────────────────┐┌───────────────┐»
q_0: ┤ Ry(0.051625) ├┤ Rz(0.0086598) ├─■─┤ Rz(-0.0086598) ├┤ Ry(-0.051625) ├»
     ├─────────────┬┘├───────────────┤ │ └────────────────┘└┬──────────────┤»
q_1: ┤ Rz(0.26461) ├─┤ Ry(-0.051652) ├─┼─────────■──────────┤ Ry(0.051652) ├»
     └────┬───┬────┘ └───────────────┘ │         │          └────┬───┬─────┘»
q_2: ─────┤ H ├────────────────────────■─────────■───────────────┤ H ├──────»
          └───┘                                                  └───┘      »
«                                   
«q_0: ──────────────────────────────
«     ┌──────────────┐┌────────────┐
«q_1: ┤ Rz(-0.26461) ├┤ U(π/2,0,π) ├
«     └──────────────┘└────────────┘
«q_2: ──────────────────────────────
«                                   

fragment 0:
       ┌──────────────┐ ┌───────────────┐                                     »
q_0: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├─────────────────────────────────────»
     ┌─┴──────────────

In [33]:
print(fragment_cuts_list[-1])

[{'prep': 1, 'meas': 1}, {'prep': 1, 'meas': 1}]


In [34]:
wire_path_map

{Qubit(QuantumRegister(5, 'q'), 0): ((0, Qubit(QuantumRegister(4, 'q'), 2)),),
 Qubit(QuantumRegister(5, 'q'), 1): ((0, Qubit(QuantumRegister(4, 'q'), 3)),),
 Qubit(QuantumRegister(5, 'q'), 2): ((1, Qubit(QuantumRegister(3, 'q'), 0)),
  (0, Qubit(QuantumRegister(4, 'q'), 0)),
  (1, Qubit(QuantumRegister(3, 'q'), 1))),
 Qubit(QuantumRegister(5, 'q'), 3): ((0, Qubit(QuantumRegister(4, 'q'), 1)),),
 Qubit(QuantumRegister(5, 'q'), 4): ((1, Qubit(QuantumRegister(3, 'q'), 2)),)}

In [35]:
num_fragments = len(fragments)
num_fragments

2

In [36]:
num_tomo_circuits = len(meas_qcs)
num_tomo_circuits

25

In [37]:
fragments[0].draw()

┌──────────────┐ ┌───────────────┐                                     »
q_0: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├─────────────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                                     »
q_1: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├──────────────────────────────────────»
     └──────┬───┬──────┘┌─┴────────────┴┐┌───────────────┐   ┌───────────────┐»
q_2: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├─■─┤ Ry(0.0082611) ├»
            ├───┤       └┬──────────────┤├───────────────┤ │ └───────────────┘»
q_3: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─■──────────────────»
            └───┘        └──────────────┘└───────────────┘                    »
«                          ┌──────────────┐ ┌──────────────┐
«q_0: ─■────────■──────────┤ Ry(0.051652) ├─┤ Rz(-0.26461) ├
«      │        │        ┌─┴──────────────┴┐└┬────────────┬┘
«q_1: ─┼────────■────────┤ Ry(-4.5556e-05) ├─┤ Rz(1.2199) ├─
«      │  ┌────────────┐ └─────────────────┘ └────────────┘ 
«q_2: ─┼──┤ Rz(1.3184) ├────────────────────────────────────
«      │ ┌┴────────────┴┐   ┌────────────┐                  
«q_3: ─■─┤ Ry(0.013556) ├───┤ Rz(1.3353) ├──────────────────
«        └──────────────┘   └────────────┘

In [38]:
fragments[1].draw()

┌──────────────┐┌───────────────┐   ┌────────────────┐┌───────────────┐»
q_0: ┤ Ry(0.051625) ├┤ Rz(0.0086598) ├─■─┤ Rz(-0.0086598) ├┤ Ry(-0.051625) ├»
     ├─────────────┬┘├───────────────┤ │ └────────────────┘└┬──────────────┤»
q_1: ┤ Rz(0.26461) ├─┤ Ry(-0.051652) ├─┼─────────■──────────┤ Ry(0.051652) ├»
     └────┬───┬────┘ └───────────────┘ │         │          └────┬───┬─────┘»
q_2: ─────┤ H ├────────────────────────■─────────■───────────────┤ H ├──────»
          └───┘                                                  └───┘      »
«                     
«q_0: ────────────────
«     ┌──────────────┐
«q_1: ┤ Rz(-0.26461) ├
«     └──────────────┘
«q_2: ────────────────
«

In [39]:
hardware_index = 0

In [40]:
print(shots)

240000


In [41]:
tomography_circuits = []
for idx in range(0, len(fragments_list)):
    fragments = fragments_list[idx]
    wire_path_map = wire_path_map_list[idx]
    temp_frag_circuits = [0] * len(fragments)
    for i in range(0, len(fragments)):
        if i == hardware_index:
            temp_frag_circuits[i] = collect_fragment_circuits(fragments[i], i, wire_path_map,
                                         shots = shots // total_variants,
                                         tomography_backend = simulation_backend)
        else:
            temp_frag_circuits[i] = collect_fragment_circuits(fragments[i], i, wire_path_map,
                                         shots = shots // total_variants,
                                         tomography_backend = simulation_backend)
    tomography_circuits.append(temp_frag_circuits)

In [42]:
#contains 34 basis
len(tomography_circuits)

25

In [43]:
#contains two fragments
len(tomography_circuits[0])

2

In [44]:
len(tomography_circuits[0][1])

1

In [45]:
#tomography circuits
len(tomography_circuits[0][1][0])

12

In [46]:
multi_exp_circuits = [[],[]]
for i in range(0, len(tomography_circuits)):
    #for each basis collect the three fragments:
    for j in range(0, num_fragments):
        fragment_circ = tomography_circuits[i][j][0]
        multi_exp_circuits[j] += fragment_circ

In [49]:
#16,14,11,8
#[11,8,16,14]

In [47]:
multi_exp_circuits[hardware_index][-200].draw()

┌─────────────────┐ ░   ┌──────────────┐ ┌───────────────┐»
  q_0: ┤ U3(-1.9106,π,0) ├─░───┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├»
       └─────────────────┘ ░ ┌─┴──────────────┴┐└─┬────────────┬┘»
  q_1: ────────────────────░─┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├─»
                           ░ └──────┬───┬──────┘┌─┴────────────┴┐»
  q_2: ────────────────────░────────┤ X ├───────┤ Ry(0.0082391) ├»
                           ░        ├───┤       └┬──────────────┤»
  q_3: ────────────────────░────────┤ X ├────────┤ Ry(0.013636) ├»
                           ░        └───┘        └──────────────┘»
c16_0: ══════════════════════════════════════════════════════════»
                                                                 »
c16_1: ══════════════════════════════════════════════════════════»
                                                                 »
c16_2: ══════════════════════════════════════════════════════════»
                                                                 »
c16_3: ══════════════════════════════════════════════════════════»
                                                                 »
«                                                               »
«  q_0: ──────────────────────────────────────■────────■────────»
«                                             │        │        »
«  q_1: ──────────────────────────────────────┼────────■────────»
«       ┌───────────────┐   ┌───────────────┐ │  ┌────────────┐ »
«  q_2: ┤ Rz(-0.010144) ├─■─┤ Ry(0.0082611) ├─┼──┤ Rz(1.3184) ├─»
«       ├───────────────┤ │ └───────────────┘ │ ┌┴────────────┴┐»
«  q_3: ┤ Rz(-0.028142) ├─■───────────────────■─┤ Ry(0.013556) ├»
«       └───────────────┘                       └──────────────┘»
«c16_0: ════════════════════════════════════════════════════════»
«                                                               »
«c16_1: ════════════════════════════════════════════════════════»
«                                                               »
«c16_2: ════════════════════════════════════════════════════════»
«                                                               »
«c16_3: ════════════════════════════════════════════════════════»
«                                                               »
«         ┌──────────────┐ ┌──────────────┐                 ░ ┌───┐      ┌─┐
«  q_0: ──┤ Ry(0.051652) ├─┤ Rz(-0.26461) ├─────────────────░─┤ H ├──────┤M├
«       ┌─┴──────────────┴┐└┬────────────┬┘┌──────────────┐ ░ └┬─┬┘      └╥┘
«  q_1: ┤ Ry(-4.5556e-05) ├─┤ Rz(1.2199) ├─┤ U(π/2,0,π/2) ├─░──┤M├────────╫─
«       └─────────────────┘ └────────────┘ └──────────────┘ ░  └╥┘ ┌─┐    ║ 
«  q_2: ────────────────────────────────────────────────────░───╫──┤M├────╫─
«          ┌────────────┐                                   ░   ║  └╥┘┌─┐ ║ 
«  q_3: ───┤ Rz(1.3353) ├───────────────────────────────────░───╫───╫─┤M├─╫─
«          └────────────┘                                   ░   ║   ║ └╥┘ ║ 
«c16_0: ════════════════════════════════════════════════════════╬═══╬══╬══╩═
«                                                               ║   ║  ║    
«c16_1: ════════════════════════════════════════════════════════╩═══╬══╬════
«                                                                   ║  ║    
«c16_2: ════════════════════════════════════════════════════════════╩══╬════
«                                                                      ║    
«c16_3: ═══════════════════════════════════════════════════════════════╩════
«

In [74]:
frag_datas = [0] * num_fragments
for i in range(0, num_fragments):
    if i == hardware_index:
        frag_datas[i] = run_circuits(multi_exp_circuits[i], initial_layout = [11,8,16,14],
                                     backend = device, shots = shots // total_variants, 
                                     optimization_level = 3, monitor_jobs = True)
    else:
        frag_datas[i] = run_circuits(multi_exp_circuits[i], initial_layout = None,
                                     backend = simulation_backend, shots = shots // total_variants,
                                     optimization_level = 3, monitor_jobs = True, noise_model = None)
       

job_id:  63918a6d44034d0f3e4d4ce6
Job Status: job has successfully run
job_id:  b3755865-0c89-4cf9-8cd8-739d737a6539
Job Status: job has successfully run


In [75]:
frag_datas[0]

[Result(backend_name='ibm_auckland', backend_version='1.3.0', qobj_id='164348ab-0c75-4519-bec1-9ee32913efe9', job_id='63918a6d44034d0f3e4d4ce6', success=True, results=[ExperimentResult(shots=10000, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 1465, '0x1': 16, '0x2': 1513, '0x3': 19, '0x4': 850, '0x5': 14, '0x6': 924, '0x7': 11, '0x8': 1547, '0x9': 18, '0xa': 1607, '0xb': 23, '0xc': 964, '0xd': 12, '0xe': 1003, '0xf': 14}), header=QobjExperimentHeader(clbit_labels=[['c0', 0], ['c0', 1], ['c0', 2], ['c0', 3]], creg_sizes=[['c0', 4]], global_phase=0.0, memory_slots=4, metadata={}, n_qubits=27, name="(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z'))", qreg_sizes=[['q', 27]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4], ['q', 5], ['q', 6], ['q', 7], ['q', 8], ['q', 9], ['q', 10], ['q', 11], ['q', 12], ['q', 13], ['q', 14], ['q', 15], ['q', 16], ['q', 17], ['q', 18], ['q', 19], ['q', 20], ['q', 21], ['q', 22], ['q', 23], ['q', 24], ['q', 25], ['q', 2

In [76]:
# from qiskit.providers.ibmq.managed import IBMQJobManager
# job_manager = IBMQJobManager()

In [77]:
# job_set = job_manager.retrieve_job_set(job_set_id='973c2fdceaf547389a770d19aad463d8-1669966038684177', provider=provider)

In [78]:
# managed_results = job_set.results()

In [79]:
frag_datas1_copy = frag_datas[1].copy()

In [80]:
frag_datas[1] = [frag_datas[1][0].combine_results()]

AttributeError: Attribute combine_results is not defined

In [ ]:
# frag_datas[0] = run_circuits(multi_exp_circuits[0], initial_layout = None,
#                              backend = simulation_backend, shots = shots // total_variants,
#                              optimization_level = 3, monitor_jobs = True, noise_model = None)

In [81]:
frag_datas[0][0].results[0].header.name

"(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z'))"

In [82]:
num_tomo_circuits

25

In [83]:
test = [[], []]

In [84]:
results_list = [[], []]
counts_list = [[], []] 
for i in range(0, num_fragments): 
    for j in range(0, num_tomo_circuits):
        start_idx = j * 12
        end_idx = (j + 1) * 12
        results_list[i].append(frag_datas[i][0].results[start_idx : end_idx])
        counts_list[i].append(frag_datas[i][0].get_counts()[start_idx : end_idx])


In [85]:
frag_datas[1][0].get_counts()

[{'101': 2,
  '110': 1,
  '100': 3,
  '001': 2,
  '011': 2,
  '111': 6,
  '000': 4925,
  '010': 5059},
 {'110': 3,
  '001': 2639,
  '100': 6,
  '010': 2387,
  '000': 2287,
  '111': 6,
  '011': 2672},
 {'101': 5,
  '111': 1,
  '011': 2580,
  '010': 2558,
  '000': 2451,
  '110': 6,
  '001': 2399},
 {'011': 3,
  '101': 4,
  '001': 4,
  '100': 3234,
  '110': 3148,
  '010': 1828,
  '000': 1779},
 {'010': 831,
  '000': 854,
  '001': 904,
  '110': 1575,
  '100': 1530,
  '011': 912,
  '111': 1712,
  '101': 1682},
 {'001': 914,
  '110': 1525,
  '100': 1719,
  '011': 829,
  '111': 1645,
  '010': 1012,
  '000': 766,
  '101': 1590},
 {'101': 7,
  '011': 1,
  '000': 1660,
  '010': 1610,
  '001': 5,
  '110': 3292,
  '100': 3425},
 {'001': 890,
  '110': 1555,
  '100': 1646,
  '111': 1766,
  '011': 845,
  '010': 844,
  '000': 697,
  '101': 1757},
 {'011': 826,
  '111': 1657,
  '001': 761,
  '100': 1661,
  '110': 1707,
  '101': 1786,
  '000': 843,
  '010': 759},
 {'111': 4,
  '011': 4,
  '010': 1644,
 

In [86]:
frag_datas[0][0].get_counts()

[{'0000': 1465,
  '0001': 16,
  '0010': 1513,
  '0011': 19,
  '0100': 850,
  '0101': 14,
  '0110': 924,
  '0111': 11,
  '1000': 1547,
  '1001': 18,
  '1010': 1607,
  '1011': 23,
  '1100': 964,
  '1101': 12,
  '1110': 1003,
  '1111': 14},
 {'0000': 735,
  '0001': 838,
  '0010': 731,
  '0011': 831,
  '0100': 442,
  '0101': 473,
  '0110': 452,
  '0111': 472,
  '1000': 700,
  '1001': 772,
  '1010': 735,
  '1011': 844,
  '1100': 465,
  '1101': 539,
  '1110': 485,
  '1111': 486},
 {'0000': 771,
  '0001': 686,
  '0010': 798,
  '0011': 760,
  '0100': 459,
  '0101': 452,
  '0110': 487,
  '0111': 462,
  '1000': 814,
  '1001': 749,
  '1010': 786,
  '1011': 799,
  '1100': 501,
  '1101': 470,
  '1110': 517,
  '1111': 489},
 {'0000': 516,
  '0001': 1028,
  '0010': 513,
  '0011': 1014,
  '0100': 300,
  '0101': 632,
  '0110': 324,
  '0111': 702,
  '1000': 467,
  '1001': 1089,
  '1010': 467,
  '1011': 1076,
  '1100': 291,
  '1101': 648,
  '1110': 273,
  '1111': 660},
 {'0000': 99,
  '0001': 1409,
  '00

In [87]:
from mlrecon_methods import organize_tomography_data_from_list
organized_data = []
for i in range(0, num_tomo_circuits):
    frag_data = []
    frag_targets = ml.identify_frag_targets(wire_path_map_list[i])
    for j in range(0, num_fragments):
        frag_data.append(organize_tomography_data_from_list(results_list[j][i], counts_list[j][i],
                                              frag_targets[j].get("prep"),
                                              frag_targets[j].get("meas"),
                                              prep_basis = "SIC", extra_qc = []))
    organized_data.append(frag_data)

(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z')) ('S0', 'S0', 'S0', 'S0') ('Z', 'Z', 'Z', 'Z')
(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z')) ('S0',) ('Z',) [0] [0]
(('S0', 'S0', 'S0', 'S0'), ('X', 'Z', 'Z', 'Z')) ('S0', 'S0', 'S0', 'S0') ('X', 'Z', 'Z', 'Z')
(('S0', 'S0', 'S0', 'S0'), ('X', 'Z', 'Z', 'Z')) ('S0',) ('X',) [0] [0]
(('S0', 'S0', 'S0', 'S0'), ('Y', 'Z', 'Z', 'Z')) ('S0', 'S0', 'S0', 'S0') ('Y', 'Z', 'Z', 'Z')
(('S0', 'S0', 'S0', 'S0'), ('Y', 'Z', 'Z', 'Z')) ('S0',) ('Y',) [0] [0]
(('S1', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z')) ('S1', 'S0', 'S0', 'S0') ('Z', 'Z', 'Z', 'Z')
(('S1', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z')) ('S1',) ('Z',) [0] [0]
(('S1', 'S0', 'S0', 'S0'), ('X', 'Z', 'Z', 'Z')) ('S1', 'S0', 'S0', 'S0') ('X', 'Z', 'Z', 'Z')
(('S1', 'S0', 'S0', 'S0'), ('X', 'Z', 'Z', 'Z')) ('S1',) ('X',) [0] [0]
(('S1', 'S0', 'S0', 'S0'), ('Y', 'Z', 'Z', 'Z')) ('S1', 'S0', 'S0', 'S0') ('Y', 'Z', 'Z', 'Z')
(('S1', 'S0', 'S0', 'S0'), ('Y', 'Z', 'Z', 'Z')) ('S1',) ('Y',) [0] [0

In [88]:
for final_bits, fixed_bit_data in frag_data[0].items():
    prep_meas_states, state_counts = zip(*fixed_bit_data.items())
    prep_labels, meas_labels = zip(*prep_meas_states)
    prep_qubit_num = len(prep_labels[0])
    meas_qubit_num = len(meas_labels[0])
    print(prep_qubit_num)

1
1
1
1
1
1
1
1


In [89]:
len(organized_data)

25

In [90]:
organized_data[2]

[{'000': {(('S0',), ('Zp',)): 2978,
   (('S0',), ('Zm',)): 35,
   (('S0',), ('Xp',)): 1397,
   (('S0',), ('Xm',)): 1651,
   (('S0',), ('Yp',)): 1610,
   (('S0',), ('Ym',)): 1393,
   (('S1',), ('Zp',)): 962,
   (('S1',), ('Zm',)): 2158,
   (('S1',), ('Xp',)): 215,
   (('S1',), ('Xm',)): 2861,
   (('S1',), ('Yp',)): 1813,
   (('S1',), ('Ym',)): 1214,
   (('S2',), ('Zp',)): 1057,
   (('S2',), ('Zm',)): 1933,
   (('S2',), ('Xp',)): 2485,
   (('S2',), ('Xm',)): 521,
   (('S2',), ('Yp',)): 2509,
   (('S2',), ('Ym',)): 512,
   (('S3',), ('Zp',)): 1185,
   (('S3',), ('Zm',)): 1880,
   (('S3',), ('Xp',)): 1976,
   (('S3',), ('Xm',)): 1128,
   (('S3',), ('Yp',)): 159,
   (('S3',), ('Ym',)): 2916},
  '001': {(('S0',), ('Zp',)): 33,
   (('S0',), ('Xp',)): 15,
   (('S0',), ('Xm',)): 10,
   (('S0',), ('Yp',)): 12,
   (('S0',), ('Ym',)): 20,
   (('S1',), ('Zp',)): 14,
   (('S1',), ('Zm',)): 21,
   (('S1',), ('Xp',)): 40,
   (('S1',), ('Xm',)): 9,
   (('S1',), ('Yp',)): 13,
   (('S1',), ('Ym',)): 22,


In [91]:
organized_data[-1][-1]

{'01': {(('S0',), ('Zm',)): 1,
  (('S0',), ('Zp',)): 9,
  (('S0',), ('Xm',)): 6,
  (('S0',), ('Xp',)): 4,
  (('S0',), ('Yp',)): 3,
  (('S0',), ('Ym',)): 3,
  (('S1',), ('Zm',)): 5,
  (('S1',), ('Zp',)): 4,
  (('S1',), ('Xp',)): 5,
  (('S1',), ('Ym',)): 3,
  (('S2',), ('Zm',)): 2,
  (('S2',), ('Zp',)): 2,
  (('S2',), ('Xm',)): 8,
  (('S2',), ('Xp',)): 3,
  (('S2',), ('Ym',)): 10,
  (('S2',), ('Yp',)): 1,
  (('S3',), ('Zm',)): 5,
  (('S3',), ('Zp',)): 3,
  (('S3',), ('Xm',)): 4,
  (('S3',), ('Yp',)): 4,
  (('S1',), ('Xm',)): 0,
  (('S1',), ('Yp',)): 0,
  (('S3',), ('Xp',)): 0,
  (('S3',), ('Ym',)): 0},
 '11': {(('S0',), ('Zp',)): 6,
  (('S0',), ('Xm',)): 4,
  (('S0',), ('Xp',)): 3,
  (('S0',), ('Ym',)): 2,
  (('S0',), ('Yp',)): 2,
  (('S1',), ('Zm',)): 4,
  (('S1',), ('Zp',)): 6406,
  (('S1',), ('Xp',)): 2995,
  (('S1',), ('Xm',)): 3385,
  (('S1',), ('Yp',)): 3211,
  (('S1',), ('Ym',)): 3199,
  (('S2',), ('Zm',)): 4,
  (('S2',), ('Zp',)): 6682,
  (('S2',), ('Xm',)): 3529,
  (('S2',), ('X

In [92]:
recombined_lists = []
filtered_recombined_lists = []
norm_filtered_lists = []
for i in range(0, num_tomo_circuits):
    direct_models = ml.direct_fragment_model(organized_data[i])
    likely_models = ml.maximum_likelihood_model(direct_models)

    direct_recombined_dist = ml.recombine_fragment_models(direct_models, wire_path_map_list[i])
    likely_recombined_dist = ml.recombine_fragment_models(likely_models, wire_path_map_list[i])
    recombined_lists.append(likely_recombined_dist)
    filtered_recombined_lists.append(filter_results(likely_recombined_dist, [0]))
    norm_filtered_lists.append(norm_dict(filter_results(likely_recombined_dist, [0])))

{'000': {(('S0',), ('Zp',)): 1465, (('S0',), ('Zm',)): 16, (('S0',), ('Xp',)): 735, (('S0',), ('Xm',)): 838, (('S0',), ('Yp',)): 771, (('S0',), ('Ym',)): 686, (('S1',), ('Zp',)): 516, (('S1',), ('Zm',)): 1028, (('S1',), ('Xp',)): 99, (('S1',), ('Xm',)): 1409, (('S1',), ('Yp',)): 924, (('S1',), ('Ym',)): 631, (('S2',), ('Zp',)): 507, (('S2',), ('Zm',)): 986, (('S2',), ('Xp',)): 1228, (('S2',), ('Xm',)): 278, (('S2',), ('Yp',)): 1213, (('S2',), ('Ym',)): 258, (('S3',), ('Zp',)): 626, (('S3',), ('Zm',)): 992, (('S3',), ('Xp',)): 998, (('S3',), ('Xm',)): 571, (('S3',), ('Yp',)): 103, (('S3',), ('Ym',)): 1429}, '001': {(('S0',), ('Zp',)): 1513, (('S0',), ('Zm',)): 19, (('S0',), ('Xp',)): 731, (('S0',), ('Xm',)): 831, (('S0',), ('Yp',)): 798, (('S0',), ('Ym',)): 760, (('S1',), ('Zp',)): 513, (('S1',), ('Zm',)): 1014, (('S1',), ('Xp',)): 119, (('S1',), ('Xm',)): 1458, (('S1',), ('Yp',)): 936, (('S1',), ('Ym',)): 655, (('S2',), ('Zp',)): 560, (('S2',), ('Zm',)): 953, (('S2',), ('Xp',)): 1207, 

{'000': {(('S0',), ('Zp',)): 2885, (('S0',), ('Zm',)): 43, (('S0',), ('Xp',)): 1407, (('S0',), ('Xm',)): 1600, (('S0',), ('Yp',)): 1607, (('S0',), ('Ym',)): 1429, (('S1',), ('Zp',)): 1047, (('S1',), ('Zm',)): 2118, (('S1',), ('Xp',)): 201, (('S1',), ('Xm',)): 2932, (('S1',), ('Yp',)): 1792, (('S1',), ('Ym',)): 1313, (('S2',), ('Zp',)): 984, (('S2',), ('Zm',)): 1912, (('S2',), ('Xp',)): 2437, (('S2',), ('Xm',)): 546, (('S2',), ('Yp',)): 2566, (('S2',), ('Ym',)): 511, (('S3',), ('Zp',)): 1208, (('S3',), ('Zm',)): 1922, (('S3',), ('Xp',)): 2030, (('S3',), ('Xm',)): 1083, (('S3',), ('Yp',)): 177, (('S3',), ('Ym',)): 2947}, '001': {(('S0',), ('Zp',)): 36, (('S0',), ('Zm',)): 1, (('S0',), ('Xp',)): 17, (('S0',), ('Xm',)): 12, (('S0',), ('Yp',)): 17, (('S0',), ('Ym',)): 17, (('S1',), ('Zp',)): 13, (('S1',), ('Zm',)): 28, (('S1',), ('Xp',)): 30, (('S1',), ('Xm',)): 6, (('S1',), ('Yp',)): 11, (('S1',), ('Ym',)): 31, (('S2',), ('Zp',)): 8, (('S2',), ('Zm',)): 20, (('S2',), ('Xp',)): 7, (('S2',),

{'11': {(('S0',), ('Zp',)): 2, (('S0',), ('Zm',)): 5, (('S0',), ('Xp',)): 4, (('S0',), ('Xm',)): 1, (('S0',), ('Ym',)): 1, (('S1',), ('Zm',)): 8, (('S1',), ('Zp',)): 3372, (('S1',), ('Xm',)): 1827, (('S1',), ('Xp',)): 1489, (('S1',), ('Yp',)): 1736, (('S1',), ('Ym',)): 1645, (('S2',), ('Zm',)): 5, (('S2',), ('Zp',)): 3446, (('S2',), ('Xm',)): 1849, (('S2',), ('Xp',)): 1722, (('S2',), ('Yp',)): 1647, (('S2',), ('Ym',)): 1861, (('S3',), ('Zp',)): 3583, (('S3',), ('Zm',)): 4, (('S3',), ('Xp',)): 1709, (('S3',), ('Xm',)): 1869, (('S3',), ('Yp',)): 1777, (('S3',), ('Ym',)): 1786, (('S0',), ('Yp',)): 0}, '00': {(('S0',), ('Zm',)): 4, (('S0',), ('Zp',)): 5324, (('S0',), ('Xm',)): 2734, (('S0',), ('Xp',)): 2537, (('S0',), ('Ym',)): 2571, (('S0',), ('Yp',)): 2642, (('S1',), ('Zp',)): 1916, (('S1',), ('Xm',)): 965, (('S1',), ('Xp',)): 960, (('S1',), ('Ym',)): 931, (('S1',), ('Yp',)): 922, (('S2',), ('Zm',)): 1, (('S2',), ('Zp',)): 1695, (('S2',), ('Xm',)): 940, (('S2',), ('Xp',)): 781, (('S2',),

{'00': {(('S0',), ('Zm',)): 6, (('S0',), ('Zp',)): 9969, (('S0',), ('Xm',)): 5276, (('S0',), ('Xp',)): 4710, (('S0',), ('Ym',)): 4993, (('S0',), ('Yp',)): 4985, (('S1',), ('Zm',)): 2, (('S1',), ('Zp',)): 3561, (('S1',), ('Xm',)): 1856, (('S1',), ('Xp',)): 1687, (('S1',), ('Ym',)): 1812, (('S1',), ('Yp',)): 1755, (('S2',), ('Zp',)): 3308, (('S2',), ('Zm',)): 1, (('S2',), ('Xm',)): 1710, (('S2',), ('Xp',)): 1565, (('S2',), ('Yp',)): 1603, (('S2',), ('Ym',)): 1688, (('S3',), ('Zp',)): 3203, (('S3',), ('Zm',)): 1, (('S3',), ('Xp',)): 1557, (('S3',), ('Xm',)): 1633, (('S3',), ('Ym',)): 1558, (('S3',), ('Yp',)): 1566}, '10': {(('S0',), ('Zm',)): 5, (('S0',), ('Xp',)): 4, (('S0',), ('Ym',)): 4, (('S0',), ('Yp',)): 2, (('S1',), ('Zm',)): 2, (('S1',), ('Zp',)): 2, (('S1',), ('Xm',)): 4, (('S1',), ('Yp',)): 3, (('S1',), ('Ym',)): 2, (('S2',), ('Zm',)): 4, (('S2',), ('Zp',)): 5, (('S2',), ('Xp',)): 3, (('S2',), ('Ym',)): 3, (('S3',), ('Zm',)): 4, (('S3',), ('Zp',)): 6, (('S3',), ('Xm',)): 2, (('S

In [93]:
# direct_models = ml.direct_fragment_model(frag_data)
# likely_models = ml.maximum_likelihood_model(direct_models)

# direct_recombined_dist = ml.recombine_fragment_models(direct_models, wire_path_map)
# likely_recombined_dist = ml.recombine_fragment_models(likely_models, wire_path_map)

In [94]:
direct_recombined_dist

{'00100': 6.821788935541708e-07,
 '10100': 0.00014760785066953152,
 '00000': 0.0005155374024010424,
 '10000': -3.349145459157891e-08,
 '00101': 2.7654891346741936e-05,
 '10101': -4.484429541856225e-05,
 '00001': 0.019976428261764278,
 '10001': -5.726177314900657e-06,
 '01101': 2.8707241988832017e-07,
 '11101': 1.7688070382320513e-05,
 '01001': 0.0003120994549053153,
 '11001': -2.2102094499106853e-07,
 '00110': -3.432115085181745e-06,
 '10110': 0.000875447775367793,
 '00010': 0.030642760681378948,
 '10010': -5.807788602949778e-06,
 '00111': -3.940058607006708e-05,
 '10111': 0.025086097466032332,
 '00011': 0.9121098444059379,
 '10011': -0.00017876131276795205,
 '01111': 2.3190856843038944e-05,
 '11111': -0.00042210141298745026,
 '01011': 0.010604838321308903,
 '11011': 3.184211722888676e-07,
 '01100': -3.5208746941065595e-08,
 '11100': 1.4610822738477387e-05,
 '01000': 1.9285591909450747e-05,
 '11000': -1.894991767150819e-08,
 '01110': 6.420882447134726e-07,
 '11110': 0.00015332934634922

In [95]:
likely_recombined_dist

{'10100': 2.3266970795590733e-05,
 '00000': 5.90656506888525e-05,
 '10101': 0.00097806075415139,
 '00001': 0.01944213368142363,
 '10110': 0.0008520182393351733,
 '00010': 0.031126935638276128,
 '10111': 0.021568855355460936,
 '00011': 0.9155193380398776,
 '11111': 0.00033655955627620414,
 '01011': 0.010023355110362918,
 '11110': 2.825553361093891e-06,
 '01010': 6.758544999072369e-05}

In [96]:
filtered_recombined_lists

[{'0000': 0.07263889687010197,
  '0100': 0.0722804627989226,
  '1000': 0.07500033929907042,
  '1100': 0.07508349895798117,
  '0001': 0.04231612094480029,
  '0101': 0.042357652176971544,
  '1001': 0.045334061246282704,
  '1101': 0.04528393487323751,
  '0010': 0.07527238364070765,
  '0110': 0.07537621116154294,
  '1010': 0.0782210135885622,
  '1110': 0.07862749548503194,
  '0011': 0.04741408329604418,
  '0111': 0.04742703043131659,
  '1011': 0.04874023946451421,
  '1111': 0.049052136535120694},
 {'0000': 0.002938360074887752,
  '0100': 0.0029562706938934965,
  '1000': 0.0026984209142539416,
  '1100': 0.002710757368039746,
  '0001': 0.0026944294724715427,
  '0101': 0.0027368240218342316,
  '1001': 0.0018844240467326867,
  '1101': 0.0018808720866253092,
  '0010': 0.12472391383831104,
  '0110': 0.12815663431093538,
  '1010': 0.12713335893741282,
  '1110': 0.13094361368504764,
  '0011': 0.10555988886606774,
  '0111': 0.10808346162183043,
  '1011': 0.1127216326828286,
  '1111': 0.115829279452

In [97]:
from utils.utils import filter_results

In [98]:
filter_direct_recombined = filter_results(direct_recombined_dist, [0])

In [99]:
filter_direct_recombined

{'0100': 6.821788935541708e-07,
 '0000': 0.0005155374024010424,
 '0101': 2.7654891346741936e-05,
 '0001': 0.019976428261764278,
 '1101': 2.8707241988832017e-07,
 '1001': 0.0003120994549053153,
 '0110': -3.432115085181745e-06,
 '0010': 0.030642760681378948,
 '0111': -3.940058607006708e-05,
 '0011': 0.9121098444059379,
 '1111': 2.3190856843038944e-05,
 '1011': 0.010604838321308903,
 '1100': -3.5208746941065595e-08,
 '1000': 1.9285591909450747e-05,
 '1110': 6.420882447134726e-07,
 '1010': 0.000171856174274892}

In [100]:

filter_likely_recombined = filter_results(likely_recombined_dist, [0])

In [101]:
filter_likely_recombined

{'0000': 5.90656506888525e-05,
 '0001': 0.01944213368142363,
 '0010': 0.031126935638276128,
 '0011': 0.9155193380398776,
 '1011': 0.010023355110362918,
 '1010': 6.758544999072369e-05}

In [102]:
norm_filter_dist = norm_dict(filter_likely_recombined)

In [103]:
norm_filter_dist

{'0000': 6.050330520473805e-05,
 '0001': 0.019915354088878217,
 '0010': 0.031884563448418786,
 '0011': 0.9378030256885093,
 '1011': 0.010267322993060896,
 '1010': 6.92304759280348e-05}

In [104]:
final_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        final_expect_val += coeff * evaluation(norm_filtered_lists[i], shots = total_counts(norm_filtered_lists[i]), Pauli = Pauli_tuple[0])  

In [105]:
final_expect_val

(-1.0517798145636694+0j)

In [106]:
import json
with open('LiH_auckland_check2.json', 'w') as f:
    json.dump(norm_filtered_lists, f)

In [107]:
mit = mthree.M3Mitigation(device)

In [108]:
mit.cals_from_file(cals_file = 'mit_data')

In [109]:
def update_dict(quasis, total_shots = None):
    for key in quasis.keys():
        quasis[key] = quasis[key]*total_shots

In [110]:
mit_lists = []
for i in range(0, len(pauli_commute)):
    total_shots = total_counts(norm_filtered_lists[i])
    quasis = mit.apply_correction(norm_filtered_lists[i], [16,14,11,8])
    update_dict(quasis, total_shots = total_shots)
    mit_lists.append(quasis)

In [111]:
mit_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        mit_val += coeff * evaluation(mit_lists[i], shots = total_counts(mit_lists[i]), Pauli = Pauli_tuple[0])  

In [112]:
mit_val

(-1.0709659961775682+0j)

In [113]:
with open('LiH_auckland_check2_mit.json', 'w') as f:
    json.dump(mit_lists, f)